This notebook has quick-and-dirty Python solutions for each day. 

The main goal here is to get the answer, by no means it's supposed to be an example of a decent Python code.

It's only minimally prettified: garbage cells are removed and data paths modified.

In [2]:
%load_ext Cython

## Day 01

In [2]:
%%time

def captcha(digits, next_char_offset=1):
    num_digits = len(digits)
    res = 0
    for i, c in enumerate(digits):
        cnext = digits[(i + next_char_offset)%num_digits]
        res += (ord(c) - ord('0'))*int(c == cnext)
    return res

def captcha2(digits):
    return captcha(digits, int(len(digits)/2))

def solution(input_file):
    with open(input_file) as f:
        input_data = f.read().strip()

    print("Part 1: {}".format(captcha(input_data)))
    print("Part 2: {}".format(captcha2(input_data)))
    
solution('day01-cython/input.txt')

Part 1: 1182
Part 2: 1152
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.4 ms


## Day02

In [3]:
%%time

def find_evenly_divisible_result(row):
    for i, x in enumerate(row):
        for y in row[i + 1:]:
            if x%y == 0:
                return int(x/y)
            elif y%x == 0:
                return int(y/x)
    return 0

def solution(input_file):
    with open(input_file) as f:
        data = [[int(x) for x in s.strip().split('\t')] 
                for s in f.readlines()]

    res1 = sum(max(row) - min(row) for row in data)    
    print("Part 1: {}".format(res1))
    
    res2 = sum(map(find_evenly_divisible_result, data))
    print("Part 2: {}".format(res2))
    
solution('day02-j/input.txt')

Part 1: 30994
Part 2: 233
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 735 µs


## Day 03

In [4]:
%%time

from itertools import islice, count

OFFS = [(1, 0), (0, -1), (-1, 0), (0, 1)]

def spiral():
    x, y = (0, 0)
    direction = 0
    step = 1
    phase = 0
    while True:
        dx, dy = OFFS[direction]
        for i in range(step):
            yield x, y
            x += dx
            y += dy
            
        if phase == 0:
            phase = 1
        else:
            phase = 0
            step += 1
        direction = (direction + 1)%4

def spiraln(n):
    return next(islice(spiral(), n - 1, n))
        
def manhattan(pos):
    return abs(pos[0]) + abs(pos[1])

def sp2(n):
    pos = list(islice(spiral(), n))[1:]
    vals = {(0, 0): 1}
    def sumvals(x, y):
        return vals.get((x + 1, y), 0) + \
            vals.get((x + 1, y - 1), 0) + \
            vals.get((x , y - 1), 0) + \
            vals.get((x - 1, y - 1), 0) + \
            vals.get((x - 1, y), 0) + \
            vals.get((x - 1, y + 1), 0) + \
            vals.get((x, y + 1), 0) + \
            vals.get((x + 1, y + 1), 0)
    for p in pos:
        if p != (0, 0):
            s = sumvals(p[0], p[1])
            vals[p] = s
    return vals[pos[-1]]

def get_spiral_val_larger(x):
    i = 2
    while True:
        if sp2(i) > x:
            return sp2(i)
        i += 1

def solution(seed):
    print("Part 1: {}".format(manhattan(spiraln(seed))))
    print("Part 2: {}".format(get_spiral_val_larger(seed)))
    
INPUT = 312051
solution(INPUT)

Part 1: 430
Part 2: 312453
CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 44.2 ms


## Day 04

In [5]:
%%time

def is_valid(passphrase, sort=False):
    words = passphrase.strip().split(' ')
    reg = set()
    for w in words:
        if sort:
            w = ''.join(sorted(w))
        if w in reg:
            return False
        reg.add(w)
    return True

def solution(input_file):
    with open(input_file) as f:
        data = f.readlines()
        
    print("Part 1: {}".format(sum(map(is_valid, data))))
    print("Part 2: {}".format(sum(map(lambda x: is_valid(x, True), data))))
    
solution('day04-perl/input.txt')

Part 1: 325
Part 2: 119
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.4 ms


## Day 05

In [6]:
%%time

def count_steps_to_exit(data):
    pos = 0
    steps = 0

    while True:
        offs = data[pos]
        data[pos] += 1
        pos += offs
        if pos < 0 or pos >= len(data):
            break
        steps += 1
        
    return steps + 1

def count_steps_to_exit2(data):
    pos = 0
    steps = 0

    while True:
        offs = data[pos]
        if offs >= 3:
            data[pos] -= 1
        else:
            data[pos] += 1
        pos += offs
        if pos < 0 or pos >= len(data):
            break
        steps += 1

    return steps + 1

def solution(input_file):
    with open(input_file) as f:
        data = [int(line.strip()) for line in f.readlines()]
        
    print('Part 1: {}'.format(count_steps_to_exit(data[:])))
    print('Part 2: {}'.format(count_steps_to_exit2(data[:])))
    
solution('day05-rebol/input.txt')

Part 1: 343467
Part 2: 24774780
CPU times: user 5.43 s, sys: 0 ns, total: 5.43 s
Wall time: 5.44 s


## Day 06

In [7]:
%%time

import numpy as np

def step(blocks):
    totalb = len(blocks)
    pos = np.argmax(blocks)
    nb = blocks[pos]
    blocks[pos] = 0        
    frac = max(1, int(nb/(totalb - 1)))
    for r in range(pos + 1, pos + totalb*2):
        blocks[r%totalb] += min(frac, nb)
        nb -= frac
        if nb < 0:
            break
    return blocks

def get_max_wrap_steps(blocks):
    blocks_copy = blocks[:]
    visited = {}
    cb = tuple(blocks_copy)
    steps = 0
    check = sum(blocks_copy)
    while cb not in visited:
        visited[cb] = steps
        cb = tuple(step(blocks_copy))
        steps += 1
        assert(check == sum(blocks_copy))
    last_step = visited[cb]
    return steps, steps - last_step

def solution(input_file):
    with open(input_file) as f:
        data = [int(line.strip()) for line in f.read().split('\t')]
    
    steps, dsteps = get_max_wrap_steps(data)
    print('Part 1: {}'.format(steps))
    print('Part 1: {}'.format(dsteps))

solution('day06-racket/input.txt')

Part 1: 12841
Part 1: 8038
CPU times: user 264 ms, sys: 268 ms, total: 532 ms
Wall time: 252 ms


## Day 07

In [8]:
%%time

def split_arg(s):
    parts = [x.strip() for x in s.split('->')]
    weight = int(parts[0][1:-1])
    children = [] if len(parts) == 1 else [x.strip() for x in parts[1].split(',')]
    return weight, children

def parse_graph(lines):
    data = [line.strip().split(' ', 1) for line in lines]
    return dict((d[0], split_arg(d[1])) for d in data)

def get_root(graph):
    non_root = set()
    for node, (_, children) in graph.items():
        for child in children:
            non_root.add(child)
    for node, _ in graph.items():
        if node not in non_root:
            return node
    return None

def get_balance_weight(graph):
    balance_weight = 0
    def get_weight(root):
        w, children = graph[root]
        ch_weights = [get_weight(ch) for ch in children]
        dw = 0
        if len(ch_weights) > 0:
            maxw = max(ch_weights)
            minw = min(ch_weights)
            if minw != maxw:
                if ch_weights.count(minw) == 1:
                    ch = children[ch_weihts.index(minw)]
                    dw = maxw - minw
                else:
                    ch = children[ch_weights.index(maxw)]
                    dw = minw - maxw
                nonlocal balance_weight
                balance_weight = graph[ch][0] + dw
        return w + sum(ch_weights) + dw
    
    get_weight(get_root(graph))
    return balance_weight

def solution(input_path):
    graph = parse_graph(open(input_path).readlines())
    
    print('Part 1: {}'.format(get_root(graph)))
    print('Part 2: {}'.format(get_balance_weight(graph)))
    
solution('day07-groovy/input.txt')

Part 1: dgoocsw
Part 2: 1275
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.81 ms


## Day 08

In [9]:
%%time

def parse_instr(line):
    parts = line.split(' ')
    sgn = 1 if parts[1] == 'inc' else -1
    return parts[0], sgn, int(parts[2]), parts[4], parts[5], int(parts[6])

def parse_file(path):
    data = [line.strip() for line in open(path).readlines()]
    return [parse_instr(line) for line in data]

def eval_program(instr):
    regs = {}
    topv = None
    for reg, op, amt, ifreg, ifop, ifamt in instr:
        ifval = regs.get(ifreg, 0)
        if (ifop == '>' and ifval > ifamt) or \
            (ifop == '<' and ifval < ifamt) or \
            (ifop == '>=' and ifval >= ifamt) or \
            (ifop == '<=' and ifval <= ifamt) or \
            (ifop == '==' and ifval == ifamt) or \
            (ifop == '!=' and ifval != ifamt):
                
            regs[reg] = regs.get(reg, 0) + op*amt
            if topv is None:
                topv = regs[reg]
            else:
                topv = max(topv, max(regs.values()))
    return max(regs.values()), topv

def solution(input_file):
    res1, res2 = eval_program(parse_file(input_file))
    print("Part 1: {}".format(res1))
    print("Part 2: {}".format(res2))
    
solution('day08-idris/input.txt')

Part 1: 5215
Part 2: 6419
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.71 ms


## Day 09

In [10]:
%%time

class Parser:
    def __init__(self, text):
        self.text = text
        self.pos = 0
        self.score = 0
        self.garbage_chars = 0

    def next_token(self):
        while(self.text[self.pos] == '!'):
            self.pos += 2
        ch = self.text[self.pos]
        self.pos += 1
        return ch

    def parse_expr(self, level = 1):
        while(self.pos < len(self.text)):
            t = self.next_token()
            if t == '}':
                break
            elif t == '<':
                while self.next_token() != '>':
                    self.garbage_chars += 1
            elif t == '{':
                self.score += level
                self.parse_expr(level + 1)
        return (self.score, self.garbage_chars)
    
def solution(input_file):
    st = open(input_file).read().strip()
    res1, res2 = Parser(st).parse_expr()
    
    print("Part 1: {}".format(res1))
    print("Part 2: {}".format(res2))
    
solution('day09-erlang/input.txt')

Part 1: 11347
Part 2: 5404
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.59 ms


## Day 10

In [33]:
%%time

def rev(nums, pos, size):
    rem = max(0, pos + size - len(nums))
    sub = list(reversed(nums[pos:(pos + size - rem)] + nums[0:rem]))
    nums[pos:(pos + size - rem)] = sub[0:size-rem]
    
    if rem > 0:
        nums[0:rem] = sub[-rem:]    
        
def transform(nums, offsets, times=1):
    cur_pos = 0
    skip_size = 0
    for i in range(times):
        for length in offsets:
            rev(nums, cur_pos, length)
            cur_pos = (cur_pos + length + skip_size)%len(nums)
            skip_size += 1
    return nums

def mogrify(text):
    return [ord(c) for c in text] + [17, 31, 73, 47, 23]

def densify(nums):
    dh = []
    for i in range(0, 256, 16):
        res = 0
        for j in range(i, i + 16):
            res ^= nums[j]
        dh.append(res)
    return dh

def stringify(sparse_nums):
    return ''.join("%0.2x"%x for x in densify(sparse_nums))

def knot_hash(seed):
    mseed = mogrify(seed)
    sparse_nums = transform(list(range(256)), mseed, 64)
    return stringify(sparse_nums)

def solution(input_path):
    seed = open(input_path).read().strip()
    offsets = [int(x) for x in seed.split(',')]
    
    res = transform(list(range(256)), offsets)
    print("Part 1: {}".format(res[0]*res[1]))
    print("Part 2: {}".format(knot_hash(seed)))
    
solution('day10-octave/input.txt')

Part 1: 37230
Part 2: 70b856a24d586194331398c7fcfa0aaf
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.8 ms


## Day 11

In [12]:
%%time

from functools import reduce 

OFFS = {
    'n': (0, -1),
    'ne': (1, 0),
    'se': (1, 1),
    's': (0, 1),
    'sw': (-1, 0),
    'nw': (-1, -1)
}

def add(t1, t2):
    return (t1[0] + t2[0], t1[1] + t2[1])

def dist(pos):
    x = pos[0]
    y = pos[1]
    if x*y <= 0:
        return abs(x) + abs(y)
    else:
        return max(abs(x), abs(y))
    
def simulate(steps):
    pos = (0, 0)
    far_dist = 0
    for s in steps:
        dpos = OFFS[s]
        pos = add(pos, dpos)
        cur_dist = dist(pos)
        if cur_dist > far_dist:
            far_dist = cur_dist
    return pos, far_dist
    
def solution(input_file):
    steps = open(input_file).read().strip().split(',')
    pos, far_dist = simulate(steps)
    
    print("Part 1: {}".format(dist(pos)))
    print("Part 2: {}".format(far_dist))
    
solution('day11-pascal/input.txt')

Part 1: 687
Part 2: 1483
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.45 ms


## Day 12

In [13]:
%%time

def parse_step(line):
    parts = line.strip().split('<->')
    p2 = [int(x) for x in parts[1].split(', ')]
    return int(parts[0]), p2

def find_index(lst, pred):
    for i, el in enumerate(lst):
        if pred(el):
            return i
    return -1

def gather_sets(steps):
    elems = set()
    for f, to in steps.items():
        elems.add(f)
        for t in to:
            elems.add(t)

    sets = [{x} for x in elems]
    for f, to in steps.items():
        for t in to:
            sf = find_index(sets, lambda s: f in s)
            st = find_index(sets, lambda s: t in s)
            if sf != st:
                # merge sets
                sets[sf] = sets[sf].union(sets[st])
                del sets[st]
    return sets

def solution(input_path):
    steps = dict(parse_step(line) for line in open(input_path).readlines())
    sets = gather_sets(steps)
    
    print("Part 1: {}".format(len(next(s for s in sets if 0 in s))))
    print("Part 2: {}".format(len(sets)))

solution('day12-r/input.txt')

Part 1: 130
Part 2: 189
CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 206 ms


## Day 13

In [14]:
%%time
%%cython


def read_data(fname):
    return [(int(parts[0]), int(parts[1])) 
            for parts in map(lambda x: x.split(":"), 
                             open(fname).readlines())]


def simulate(desc, delay=0, compute_severity=True):
    severity = 0
    n = len(desc)
    state = [delay for k in range(n)]
    
    last_step = -1
    caught = False
    for i in range(n):
        step, rng = desc[i]
        delta = step - last_step
        for j in range(i, n):
            state[j] += delta
        if state[i]%(rng*2 - 2) == 1:
            caught = True
            if not compute_severity:
                return True, 0
            severity += step*rng

        last_step = step
        
    return caught, severity


def find_min_delay(data):
    delay = 0
    while True:
        caught, sev = simulate(data, delay, False)
        if not caught:
            break
        delay += 1
    return delay


def solution(input_path):
    data = read_data(input_path)
    
    print('Part 1: {}'.format(simulate(data)[1]))
    print('Part 2: {}'.format(find_min_delay(data)))

solution('day13-fortran/input.txt')

Part 1: 1504
Part 2: 3823370
CPU times: user 10.7 s, sys: 8 ms, total: 10.7 s
Wall time: 10.7 s


## Day 14

In [37]:
%%time

def build_locations(seed):
    locs = {}
    for i in range(128):
        cur_seed = "{}-{}".format(seed, i)
        h = knot_hash(cur_seed)
        for j in range(4):
            start = j*8        
            hp = h[start:(start + 8)]
            k = int(hp, 16)
            bk = "{:032b}".format(k)            
            for x, c in enumerate(bk):
                if c == '1':
                    locs[(start*4 + x, i)] = 0
    return locs


def count_regions(locs):
    cur_reg = 1
    def flood(loc, val):
        if loc in locs and locs[loc] == 0:
            locs[loc] = val
            x, y = loc
            flood((x + 1, y), val)
            flood((x - 1, y), val)
            flood((x, y - 1), val)
            flood((x, y + 1), val)

    for loc in locs:
        if locs[loc] == 0:
            flood(loc, cur_reg)
            cur_reg += 1
    return cur_reg - 1


def solution(input_path):
    seed = open(input_path).read().strip()
    locs = build_locations(seed)
    print('Part 1: {}'.format(len(locs)))
    print('Part 2: {}'.format(count_regions(locs)))
    

solution('day14-coffeescript/input.txt')

Part 1: 8222
Part 2: 1086
CPU times: user 296 ms, sys: 0 ns, total: 296 ms
Wall time: 294 ms


## Day 15

In [6]:
%%time
%%cython

MUL1 = 16807
MUL2 = 48271
DIV = 2147483647

MASK = 0xFFFF
MASK1 = 4 - 1
MASK2 = 8 - 1       

def step1(pos):
    return ((pos[0]*MUL1)%DIV, (pos[1]*MUL2)%DIV)

def step2(pos):
    x = pos[0]
    while True:
        x = (x*MUL1)%DIV
        if x&MASK1 == 0:
            break
    y = pos[1]
    while True:
        y = (y*MUL2)%DIV
        if y&MASK2 == 0:
            break
    return (x, y)

def count_ones(start, steps, stepfn):
    num_ones = 0
    pos = start
    for i in range(steps):
        pos = stepfn(pos)
        num_ones += int(pos[0]&MASK == pos[1]&MASK)
    return num_ones
    
def solution(input_path):
    import re
    start = tuple(int(x) for x in re.findall(r'starts with (\d+)', 
                                             open(input_path).read()))

    print('Part 1: {}'.format(count_ones(start, 40000000, step1)))
    print('Part 2: {}'.format(count_ones(start, 5000000, step2)))

solution('day15-assembly/input.txt')

Part 1: 612
Part 2: 285
CPU times: user 33.8 s, sys: 0 ns, total: 33.8 s
Wall time: 33.8 s


## Day 16

In [17]:
%%time

def rotate(arr, n):
    return arr[-n:] + arr[:-n]

def dance(state, steps):
    for s in steps:
        if s[0] == 's':
            state = rotate(state, int(s[1:]))
        elif s[0] == 'x':
            ap, bp = [int(x) for x in s[1:].split("/")]
            state[bp], state[ap] = state[ap], state[bp]
        elif s[0] == 'p':
            a, b = [ord(x) - ord('a') for x in s[1:].split("/")]
            ap, bp = state.index(a), state.index(b) 
            state[ap], state[bp] = b, a
    return state

def render(programs):
    return ''.join(chr(ord('a') + x) for x in programs)

def find_period(state, steps):
    s = state[:]
    visited = {}
    i = 0
    while True:
        st = tuple(s)
        if st in visited:
            return i, i - visited[st]
        visited[st] = i
        i += 1
        s = dance(s, steps)

def dance_long(state, steps, n):
    s = state[:]
    for i in range(n):
        s = dance(s, steps)
    return s

STEPS = 1000000000

def solution(input_path):
    state = list(range(16))
    steps = open(input_path).read().strip().split(',')

    print('Part 1: {}'.format(render(dance(state[:], steps))))
    
    period, period_start = find_period(state, steps)
    state1 = dance_long(state, steps, (STEPS - period_start)%period)
    print('Part 2: {}'.format(render(state1)))

solution('day16-processing/solution/input.txt')

Part 1: ceijbfoamgkdnlph
Part 2: pnhajoekigcbflmd
CPU times: user 956 ms, sys: 0 ns, total: 956 ms
Wall time: 957 ms


## Day 17

In [18]:
%%time

def spin(step, top):
    buf = [0]
    pos = 0
    for i in range(1, top + 1):
        pos = (pos + step + 1)%len(buf)
        buf.insert(pos + 1, i)
    return buf

def spin_no_buf(step, top):
    res, pos, size = 0, 0, 1
    for i in range(1, top + 1):
        pos = (pos + step + 1)%size
        if pos == 0:
            res = i
        size += 1
    return res
    
TOP1 = 2017
TOP2 = 50000000
    
def solution(input_path):
    step = int(open(input_path).read())
    res = spin(step, TOP1)
    print("Part 1: {}".format(res[res.index(TOP1) + 1]))
    print("Part 2: {}".format(spin_no_buf(step, TOP2)))

solution("day17-visualbasic/input.txt")

Part 1: 772
Part 2: 42729050
CPU times: user 6.04 s, sys: 0 ns, total: 6.04 s
Wall time: 6.04 s



## Day 18

In [19]:
%%time

def run(commands):
    freq = None
    regs = [0]*(ord('z') - ord('a'))
    
    def get_reg(r):
        return regs[ord(r[0]) - ord('a')]
    def set_reg(r, val):
        regs[ord(r[0]) - ord('a')] = val
        
    def val(x):
        if isinstance(x, int):
            return x
        return get_reg(x)
    
    pos = 0
    while True:
        cmd = commands[pos]
        c = cmd[0]
        if c == 'snd':
            freq = val(cmd[1])
        elif c == 'set':
            set_reg(cmd[1], val(cmd[2]))
        elif c == 'add':
            set_reg(cmd[1], val(cmd[2]) + val(cmd[1]))
        elif c == 'mul':
            set_reg(cmd[1], val(cmd[2])*val(cmd[1]))            
        elif c == 'mod':
            set_reg(cmd[1], val(cmd[1])%val(cmd[2]))            
        elif c == 'rcv':
            f = val(cmd[1])
            if f != 0:
                return freq
        elif c == 'jgz':
            x = val(cmd[1])
            if x > 0:
                pos += val(cmd[2]) - 1
        pos += 1
        
        
def run_parralel(commands, pid, mailbox, snd_cnt):
    freq = None
    regs = [0]*(ord('z') - ord('a'))    
    
    def get_reg(r):
        return regs[ord(r[0]) - ord('a')]
    def set_reg(r, val):
        regs[ord(r[0]) - ord('a')] = val
     
    def val(x):
        if isinstance(x, int):
            return x
        return get_reg(x)
    
    def get_mailbox(r):
        if len(mailbox[1 - pid]) > 0:
            set_reg(r, mailbox[1 - pid][0])
            del mailbox[1 - pid][0]
            return True
        else:
            return False
 
    set_reg('p', pid)

    pos = 0
    while True:
        cmd = commands[pos]
        c = cmd[0]
        if c == 'snd':
            mailbox[pid].append(val(cmd[1]))
            snd_cnt[pid] += 1
        elif c == 'set':
            set_reg(cmd[1], val(cmd[2]))
        elif c == 'add':
            set_reg(cmd[1], val(cmd[2]) + val(cmd[1]))
        elif c == 'mul':
            set_reg(cmd[1], val(cmd[2])*val(cmd[1]))            
        elif c == 'mod':
            set_reg(cmd[1], val(cmd[1])%val(cmd[2]))            
        elif c == 'rcv':
            if not get_mailbox(cmd[1]):
                yield
                if not get_mailbox(cmd[1]):
                    # deadlock
                    return
        elif c == 'jgz':
            x = val(cmd[1])
            if x > 0:
                pos += val(cmd[2]) - 1
        pos += 1
        
        
def part2(commands):
    mailbox = [[], []]
    snd_cnt = [0, 0]

    p1 = run_parralel(commands, 0, mailbox, snd_cnt)
    p2 = run_parralel(commands, 1, mailbox, snd_cnt)

    while True:
        try:
            next(p1)
            next(p2)
        except StopIteration:
            break  
    return snd_cnt[1]
                
    
def to_int(cmd):
    for i, c in enumerate(cmd):
        try:
            cmd[i] = int(c)
        except:
            pass
    return cmd


def solution(input_path):
    commands = [to_int(x.strip().split(' ')) for x in open(input_path)]
    print('Part 1: {}'.format(run(commands)))    
    print('Part 2: {}'.format(part2(commands)))    

solution('day18-pony/input.txt')

Part 1: 4601
Part 2: 6858
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 108 ms


## Day 19

In [20]:
%%time

OFFS = [(-1, 0), (0, -1), (1, 0), (0, 1)]

def traverse(maze, start):
    pos = start
    res = ''
    dx, dy = 0, 1
    steps = 1
    while True:
        x0, y0 = pos
        x, y = x0 + dx, y0 + dy
        steps += 1

        c = maze[y][x]
        found = True
        if c not in '-|':
            found = False
            for ox, oy in OFFS:
                x1, y1 = x + ox, y + oy
                if maze[y1][x1] != ' ' and (x1, y1) != pos:
                    dx, dy = ox, oy
                    found = True
                    break
        if c not in '+-|':
            res += c
        if not found:
            return (res, steps)
        pos = (x, y)


def solution(input_path):
    maze = open(input_path).readlines()
    start = (maze[0].index('|'), 0)
    hops, steps = traverse(maze, start)
    print('Part 1: {}'.format(hops))
    print('Part 2: {}'.format(steps))
    
solution('day19-forth/input.txt')

Part 1: DWNBGECOMY
Part 2: 17228
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.61 ms


## Day 20

In [21]:
%%time
%%cython

import numpy as np
import re
from collections import namedtuple

Particle = namedtuple('Particle', 'p v a')


def parse_line(line):
    def get_vec(v):
        return tuple(map(int, v.split(',')))
    parts = dict(re.findall('(.)=<([^>]+)>', line))
    return Particle(p=get_vec(parts['p']),
                    v=get_vec(parts['v']),
                    a=get_vec(parts['a']))


def add(p1, p2):
    return (p1[0] + p2[0], p1[1] + p2[1], p1[2] + p2[2])


def magnitude(p):
    return abs(p[0]) + abs(p[1]) + abs(p[2])


def step(particles, active):
    for i, particle in enumerate(particles):
        if active[i]:
            v = add(particle.v, particle.a)
            p = add(particle.p, v)
            particles[i] = Particle(p=p, v=v, a=particle.a)
            
        
def collide(particles, active):
    pmap = {}
    for i, particle in enumerate(particles):
        if active[i]:
            if particle.p in pmap:
                active[i] = False
                active[pmap[particle.p]] = False
            else:
                pmap[particle.p] = i

                
def find_closest(particles):                
    return np.argmin([magnitude(p.p)for p in particles])


MAX_ITER = 1000000
PLATEAU_SIZE = 1000

def converge(particles, do_collision=False):
    active = [True]*len(particles)
    last_nactive, last_closest = -1, -1
    for i in range(MAX_ITER):
        step(particles, active)
        nactive, closest = -1, -1
        if do_collision:
            collide(particles, active)
            nactive = sum(active)
        else:
            closest = find_closest(particles)
            
        if (i + 1)%PLATEAU_SIZE == 0:
            if last_nactive == nactive and last_closest == closest:
                return closest, nactive
            last_nactive = nactive
            last_closest = closest
        
        
def solution(input_path):    
    particles = [parse_line(line) for line in open(input_path).readlines()]
    print('Part 1: {}'.format(converge(particles[:], False)[0]))
    print('Part 2: {}'.format(converge(particles[:], True)[1]))
    
    
solution('day20-ocaml/input.txt')

Part 1: 364
Part 2: 420
CPU times: user 3.97 s, sys: 12 ms, total: 3.98 s
Wall time: 5.12 s


## Day 21

In [22]:
%%time

import math

def get_side(s):
    return int(round(math.sqrt(len(s))))


def idx(x, y, side):
    return x + y*side


def flip(s, side):
    res = [' ']*len(s)
    for x in range(side):
        for y in range(side):
            res[idx(side - x - 1, y, side)] = s[idx(x, y, side)]
    return ''.join(res)


def rot(s, side):
    res = [' ']*len(s)
    for x in range(side):
        for y in range(side):
            res[idx(side - y - 1, x, side)] = s[idx(x, y, side)]
    return ''.join(res)


def expand_rule(rule):
    side = get_side(rule)
    
    rot1 = rot(rule, side)
    rot2 = rot(rot1, side)
    rot3 = rot(rot2, side)
    
    return [rule, rot1, rot2, rot3, 
            flip(rule, side), flip(rot1, side), 
            flip(rot2, side), flip(rot3, side)]


def parse_rules(path):
    for line in open(path).readlines():
        parts = [p.strip().replace('/', '') 
                 for p in line.strip().split('=>')]
        for r in expand_rule(parts[0]):
            yield r, parts[1]
            
    
def blit(square_side, 
         src, x_src, y_src, side_src, 
         dst, x_dst, y_dst, side_dst):
    for i in range(square_side):
        for j in range(square_side):
            dst[idx(x_dst + i, y_dst + j, side_dst)] = \
                src[idx(x_src + i, y_src + j, side_src)]

            
def rewrite(rules, state, s0, s1):
    side0 = get_side(state)
    num_sq = side0//s0
    side1 = num_sq*s1
    
    res = [' ']*side1*side1
    buf = [' ']*s0*s0
    
    for x in range(0, num_sq):
        for y in range(0, num_sq):
            blit(s0,
                 state, x*s0, y*s0, side0, 
                 buf, 0, 0, s0)            
            
            target = rules[''.join(buf)]
            assert(len(target) == s1*s1)
            
            blit(s1,
                 target, 0, 0, s1, 
                 res, x*s1, y*s1, side1) 
    return ''.join(res)


def step(rules, state):
    side = get_side(state)
    res = []
    if side%2 == 0:
        return rewrite(rules, state, 2, 3)
    elif side%3 == 0:
        return rewrite(rules, state, 3, 4)
    else:
        assert(False)
    
    
def iter_state(rules, state, steps):
    for i in range(steps):
        state = step(rules, state)
    return state


def count_on(state):
    return state.count('#')


START = '.#.'\
        '..#'\
        '###'

def solution(input_path):
    rules = dict(parse_rules(input_path))
    
    print('Part 1: {}'.format(count_on(iter_state(rules, START, 5))))
    print('Part 2: {}'.format(count_on(iter_state(rules, START, 18))))

    
solution('day21-eiffel/input.txt')

Part 1: 162
Part 2: 2264586
CPU times: user 6.56 s, sys: 12 ms, total: 6.58 s
Wall time: 6.58 s


## Day 22

In [23]:
%%time

DIRS = [(0, -1), (1, 0), (0, 1), (-1, 0)]


def turn_back(d):
    return (d + 2)%4


def turn_right(d):
    return (d + 1)%4


def turn_left(d):
    return (d - 1)%4


def add(p1, p2):
    return (p1[0] + p2[0], p1[1] + p2[1])


def step(pos, cur_dir, state):
    infected = False
    if pos in state:
        del state[pos]
        cur_dir = turn_right(cur_dir)
    else:
        state[pos] = '#'
        cur_dir = turn_left(cur_dir)
        infected = True
    pos = add(pos, DIRS[cur_dir])
    return pos, cur_dir, infected


def step2(pos, cur_dir, state):
    infected = False
    if pos not in state:
        state[pos] = 'W'
        cur_dir = turn_left(cur_dir)
    elif state[pos] == 'W':
        state[pos] = '#'
        infected = True
    elif state[pos] == '#':
        state[pos] = 'F'
        cur_dir = turn_right(cur_dir)
    elif state[pos] == 'F':
        del state[pos]
        cur_dir = turn_back(cur_dir)
    pos = add(pos, DIRS[cur_dir])
    return pos, cur_dir, infected


def step_many(state, steps, update_fn=step, 
              start_pos=(0, 0), start_dir=0):
    cpos, cdir = start_pos, start_dir
    total_infected = 0
    for i in range(steps):
        cpos, cdir, infected = update_fn(cpos, cdir, state)
        total_infected += int(infected)
    return total_infected


def init_state(layout):
    assert(len(layout) == len(layout[0]))
    coffs = len(layout[0])//2
    state = {}
    for y, line in enumerate(layout):
        for x, c in enumerate(line):
            if c == '#':
                state[(x - coffs, y - coffs)] = '#'
    return state


def solution(input_path):
    layout = [line.strip() for line in open(input_path).readlines()]
    
    res1 = step_many(init_state(layout), 10000)
    print('Part 1: {}'.format(res1))
    
    res2 = step_many(init_state(layout), 10000000, step2)
    print('Part 2: {}'.format(res2))

    
solution('day22-tcl/input.txt')

Part 1: 5352
Part 2: 2511475
CPU times: user 9.92 s, sys: 8 ms, total: 9.93 s
Wall time: 9.94 s


## Day 23

In [1]:
%%time

from collections import defaultdict

def to_int(cmd):
    for i, c in enumerate(cmd):
        try:
            cmd[i] = int(c)
        except:
            pass
    return cmd


def parse_commands(input_path):
    return [to_int(x.strip().split(' ')) for x in open(input_path)]


LAST_REG = 'h'
def run(commands, init_regs=None, count_calls=False, max_instr=-1):
    call_count = defaultdict(int)
    
    regs = [0]*(ord(LAST_REG) - ord('a') + 1)
    
    def get_reg(r):
        return regs[ord(r[0]) - ord('a')]
    def set_reg(r, val):
        regs[ord(r[0]) - ord('a')] = val

    if init_regs is not None:
        for r, val in init_regs.items():
            set_reg(r, val)

    def val(x):
        if isinstance(x, int):
            return x
        return get_reg(x)
    
    pos = 0
    i = 0
    while pos >= 0 and pos < len(commands):
        if max_instr >= 0 and i == max_instr:
            break

        cmd = commands[pos]
        c = cmd[0]
        
        if c == 'set':
            set_reg(cmd[1], val(cmd[2]))
        elif c == 'sub':
            set_reg(cmd[1], val(cmd[1]) - val(cmd[2]))
        elif c == 'mul':
            set_reg(cmd[1], val(cmd[2])*val(cmd[1]))    
        elif c == 'jnz':
            x = val(cmd[1])
            if x != 0:
                pos += val(cmd[2]) - 1
        
        call_count[c] += 1        
        pos += 1
        i += 1
        
    regs_res = {}
    for i, val in enumerate(regs):
        regs_res[chr(i + ord('a'))] = val
    return regs_res, call_count


def is_prime(n):
    k = 2
    while k*k <= n:
        if n%k == 0:
            return False
        k += 1
    return True


def manually_optimized_proc(b, c, step):
    """We somehow manually inferred form the input program that it 
    always counts the number of non-primes in form
    x = b + kP, x <= c, k = 0,1,2,3...
    and expect the input program to only differ in terms of (b, c, P)
    """
    r = range(b, c + 1, step)
    return len(r) - len(list(filter(is_prime, r)))


def run_part2(commands):
    regs, _ = run(commands, {'a': 1}, max_instr=100)
    b = regs['b']
    c = regs['c']
    inc_b = [cmd[2] for i, cmd in enumerate(commands) 
             if cmd[0] == 'sub' and cmd[1] == 'b']
    if len(inc_b) == 0:
        raise ValueError('Input program must have a sub'\
                         ' instruction for register b')
    step = -inc_b[-1]
    return manually_optimized_proc(b, c, step)


def solution(input_path):
    commands = parse_commands(input_path)
    regs, counts = run(commands, count_calls=True)
    print('Part 1: {}'.format(counts['mul']))
    
    res = run_part2(commands)
    print('Part 2: {}'.format(res))

    
solution('day23-haxe/input.txt')

Part 1: 3025
Part 2: 915
CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 29.6 ms


## Day 24

In [25]:
%%time
from collections import defaultdict


def find_best(ports, by_strength=False):
    port_reg = defaultdict(list)
    for i, (a, b) in enumerate(ports):
        port_reg[a].append(i)
        port_reg[b].append(i)

    used = [False]*len(ports)
    
    def best_subchain(pin):
        max_strength = 0
        max_chain = []
        
        def is_better(strength, chain_len):
            if by_strength:
                return strength > max_strength
            else:
                return chain_len > len(max_chain) or \
                    (chain_len == len(max_chain) and \
                     strength > max_strength)
        
        free_ports = (k for k in port_reg[pin] if not used[k])
        for pidx in free_ports:
            used[pidx] = True
            a, b = ports[pidx]
            free_pin = a if b == pin else b
            strength, chain = best_subchain(free_pin)
            strength += a + b
            if is_better(strength, len(chain) + 1):
                max_strength = strength
                max_chain = [(a, b)] + chain                    
            used[pidx] = False
        return max_strength, max_chain
    
    strength, chain = best_subchain(0)
    return strength


def solution(input_path):
    ports = [tuple(map(int, line.strip().split('/'))) 
             for line in open(input_path).readlines()]
    
    print('Part 1: {}'.format(find_best(ports, True)))
    print('Part 2: {}'.format(find_best(ports, False)))

solution('day24-pharo/input.txt')

Part 1: 1695
Part 2: 1673
CPU times: user 4.75 s, sys: 0 ns, total: 4.75 s
Wall time: 4.75 s


## Day 25

In [26]:
%%time

import re
from collections import namedtuple, defaultdict

StateBlock = namedtuple('StateBlock', 'state if0 if1')
IfBlock = namedtuple('IfBlock', 'write move target_state')


def parse_if_block(ifc):
    write = int(re.findall(r'Write the value (.)\.', ifc)[0])
    move = re.findall(r'slot to the (.+)\.', ifc)[0]
    move_dir = 1 if move == 'right' else -1
    state = re.findall(r'Continue with state (.)\.', ifc)[0]
    return IfBlock(write, move_dir, state)


def parse_state_block(block):
    ifs = block.split('If the current value is ')
    state = re.findall(r'(.):', ifs[0])[0]
    
    return StateBlock(state, 
                      parse_if_block(ifs[1]), 
                      parse_if_block(ifs[2]))


def checksum(tape):
    return sum(k for k in tape.values() if k == 1)


def eval_program(program, start_state, steps):
    state = start_state
    pos = 0
    tape = defaultdict(int)
    for i in range(steps):
        val = tape[pos]
        block = program[state]
        ifb = block.if0 if val == 0 else block.if1
        tape[pos] = ifb.write
        pos += ifb.move
        state = ifb.target_state
    return checksum(tape)


def solution(input_path):
    text = open(input_path).read()

    start_state = re.findall(r'Begin in state (.)', text)[0]
    steps = int(re.findall(r'checksum after (.+) steps', text)[0])
    state_blocks = [parse_state_block(block) 
                    for block in text.split('In state ')[1:]]
    program = {b.state: b for b in state_blocks}
    
    res = eval_program(program, start_state, steps)
    
    print('Part 1: {}'.format(res))
    print('Part 2: -')
    
solution('day25-c/input.txt')

Part 1: 4385
Part 2: -
CPU times: user 4.36 s, sys: 0 ns, total: 4.36 s
Wall time: 4.37 s
